In [56]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.DataFrame({
    'startSleepDate': ['2023-12-11 16:28:51', '2023-12-12 14:34:51', '2023-12-13 14:48:51',
                       '2023-12-14 15:38:51', '2023-12-15 15:37:51', '2023-12-16 15:14:51',
                       '2023-12-17 16:01:51', '2023-12-18 16:29:51', '2023-12-19 16:03:51',
                       '2023-12-20 14:32:51', '2023-12-21 13:44:51', '2023-12-22 14:44:51',
                       '2023-12-23 15:43:51', '2023-12-24 14:30:51', '2023-12-25 15:08:51',
                       '2023-12-26 16:28:51', '2023-12-27 15:42:51', '2023-12-28 14:49:51',
                       '2023-12-29 15:01:51', '2023-12-30 15:47:51'],
    'endSleepDate': ['2023-12-12 00:28:51', '2023-12-12 21:34:51', '2023-12-13 20:48:51',
                     '2023-12-14 23:38:51', '2023-12-15 21:37:51', '2023-12-16 22:14:51',
                     '2023-12-17 22:01:51', '2023-12-18 22:29:51', '2023-12-19 23:03:51',
                     '2023-12-20 21:32:51', '2023-12-21 21:44:51', '2023-12-22 20:44:51',
                     '2023-12-23 21:43:51', '2023-12-24 20:30:51', '2023-12-25 22:08:51',
                     '2023-12-27 00:28:51', '2023-12-27 23:42:51', '2023-12-28 22:49:51',
                     '2023-12-29 23:01:51', '2023-12-30 23:47:51']
})

# 날짜/시간 변환
df['startSleepDate'] = pd.to_datetime(df['startSleepDate'])
df['endSleepDate'] = pd.to_datetime(df['endSleepDate'])

# 수면 기간 계산 (시간 단위)
df['sleepDuration'] = (df['endSleepDate'] - df['startSleepDate']).dt.total_seconds() / 3600
df.loc[df['sleepDuration'] < 0, 'sleepDuration'] += 24

# 수면 시작 시간을 시간의 소수점 형식으로 변환 (예: 16:30 -> 16.5)
df['start_hour'] = df['startSleepDate'].dt.hour + df['startSleepDate'].dt.minute / 60 + df['startSleepDate'].dt.second / 3600

# 각 수면 이벤트에 대한 ARIMA 모델
model_start_time = ARIMA(df['start_hour'], order=(1, 1, 1))
model_start_time_fit = model_start_time.fit()
start_time_forecast = model_start_time_fit.forecast(steps=5).reset_index(drop=True)
start_time_forecast




0    16.019658
1    15.962518
2    15.977215
3    15.973434
4    15.974407
Name: predicted_mean, dtype: float64

In [55]:
model_duration= ARIMA(df['sleepDuration'], order=(1, 1, 1))
model_duration_fit = model_duration.fit()
duration_forecast = model_duration_fit.forecast(steps=5).reset_index(drop=True)
duration_forecast

0    7.770614
1    7.737604
2    7.732854
3    7.732170
4    7.732072
Name: predicted_mean, dtype: float64

In [57]:
# 예측된 시작 시간과 기간을 사용하여 종료 시간 계산
future_dates = pd.date_range(start=df['startSleepDate'].max(), periods=6, freq='D')[1:]
final_predictions = pd.DataFrame({
    'Predicted Start Time': [convert_to_datetime(future_dates[i], start_time_forecast[i]) for i in range(5)],
    'Predicted End Time': [convert_to_datetime(future_dates[i], start_time_forecast[i]) + timedelta(hours=duration_forecast[i]) for i in range(5)]
}, index=future_dates)

final_predictions

,Predicted Start Time,Predicted End Time
2023-12-31 15:47:51,2023-12-31 16:01:51,2023-12-31 23:48:05.209083
2024-01-01 15:47:51,2024-01-01 15:57:51,2024-01-01 23:42:06.374172
2024-01-02 15:47:51,2024-01-02 15:58:51,2024-01-02 23:42:49.273311
2024-01-03 15:47:51,2024-01-03 15:58:51,2024-01-03 23:42:46.812422
2024-01-04 15:47:51,2024-01-04 15:58:51,2024-01-04 23:42:46.458290


In [48]:
# 각 수면 이벤트에 대한 ARIMA 모델
model_start_time = ARIMA(df['start_hour'], order=(1, 1, 1))
model_start_time_fit = model_start_time.fit()
start_time_forecast = model_start_time_fit.forecast(steps=5)

# 각 수면 이벤트에 대한 수면 시작 시간 및 기간 예측
start_time_forecast = model_start_time_fit.forecast(steps=5).reset_index(drop=True)
duration_forecast = model_duration_fit.forecast(steps=5).reset_index(drop=True)

# 예측된 시작 시간과 기간을 사용하여 종료 시간 계산
future_dates = pd.date_range(start=df['startSleepDate'].max(), periods=6, freq='D')[1:]
final_predictions = pd.DataFrame({
    'Predicted Start Time': [convert_to_datetime(future_dates[i], start_time_forecast[i]) for i in range(5)],
    'Predicted End Time': [convert_to_datetime(future_dates[i], start_time_forecast[i]) + timedelta(hours=duration_forecast[i]) for i in range(5)]
}, index=future_dates)

# 최종 예측 결과 출력
print(final_predictions)

                    Predicted Start Time         Predicted End Time
2023-12-31 15:47:51  2023-12-31 16:01:51 2023-12-31 23:48:05.209083
2024-01-01 15:47:51  2024-01-01 15:57:51 2024-01-01 23:42:06.374172
2024-01-02 15:47:51  2024-01-02 15:58:51 2024-01-02 23:42:49.273311
2024-01-03 15:47:51  2024-01-03 15:58:51 2024-01-03 23:42:46.812422
2024-01-04 15:47:51  2024-01-04 15:58:51 2024-01-04 23:42:46.458290
